In [1]:
!pip install -r https://raw.githubusercontent.com/bentoml/BentoML/main/examples/quickstart/requirements.txt

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 873 kB 5.0 MB/s 
     |████████████████████████████████| 135 kB 35.5 MB/s 
     |████████████████████████████████| 52 kB 545 kB/s 
     |████████████████████████████████| 64 kB 1.3 MB/s 
     |████████████████████████████████| 56 kB 3.2 MB/s 
     |████████████████████████████████| 50 kB 4.8 MB/s 
     |████████████████████████████████| 237 kB 25.6 MB/s 
     |████████████████████████████████| 81 kB 7.8 MB/s 
     |████████████████████████████████| 182 kB 9.7 MB/s 
     |████████████████████████████████| 133 kB 43.2 MB/s 
     |████████████████████████████████| 46 kB 3.7 MB/s 
     |████████████████████████████████| 1.2 MB 39.5 MB/s 
     |████████████████████████████████| 2.1 MB 31.2 MB/s 
     |████████████████████████████████| 51 kB 5.8 MB/s 
     |████████████████████████████████| 80 kB 6.6 MB/s 
     |████████████████████████████████| 58 kB 4.9

In [2]:
import pandas as pd
import numpy as np

In [3]:
df1 = pd.read_csv('https://raw.githubusercontent.com/arinaaandreeva/Datasets/main/CO2_emission_2008_13.csv')
df2 = pd.read_csv('https://raw.githubusercontent.com/arinaaandreeva/Datasets/main/CO2_emission_2014.csv')
df = pd.concat([df1, df2], axis=0)
print(df1.shape, df2.shape, df.shape)

(1584, 13) (264, 13) (1848, 13)


In [4]:
df.columns = df.columns.str.lower().str.replace(' ', '_')

In [5]:
strings = list(df.dtypes[df.dtypes == 'object'].index)
strings = strings[1:len(strings)]
strings

['gdp_growth',
 'co2_emission_metrics_per_capita',
 'electricity_access',
 'urban_population',
 'population_growth',
 'urban_population_growth_annual',
 'live_stock_production',
 'forest_area',
 'renewable_energy_use',
 'energy_use_kg_of_oil_per_capita',
 'population_total']

In [6]:
for i in strings:
  df[i]=df[i].apply(lambda x:x.replace('..',''))
  df[i]=df[i].apply(lambda x:x.replace(',',''))
  df[i]=df[i].apply(pd.to_numeric)

df['year']=df['year'].astype(object)
df = df.fillna(0)

In [7]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import mean_squared_error as MSE
from sklearn.metrics import r2_score

In [8]:
ohe = OneHotEncoder()
transformed = ohe.fit_transform(df[['country', 'year']])
print(transformed.toarray())

[[1. 0. 0. ... 0. 0. 0.]
 [0. 1. 0. ... 0. 0. 0.]
 [0. 0. 1. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 1.]
 [0. 0. 0. ... 0. 0. 1.]
 [0. 0. 0. ... 0. 0. 1.]]


In [9]:
df_full_train, df_test = train_test_split(df, test_size=0.2, random_state=1)
df_train, df_val = train_test_split(df_full_train, test_size=0.25, random_state=1)

In [10]:
y_train = df_train.co2_emission_metrics_per_capita.values
y_val = df_val.co2_emission_metrics_per_capita.values
y_test = df_test.co2_emission_metrics_per_capita.values

del df_train['co2_emission_metrics_per_capita']
del df_val['co2_emission_metrics_per_capita']
del df_test['co2_emission_metrics_per_capita']

In [11]:
from sklearn.feature_extraction import DictVectorizer
dv = DictVectorizer(sparse=False)

X_train = df_train.reset_index(drop=True)
X_train_dict = X_train.to_dict(orient='records')
X_train = dv.fit_transform(X_train_dict)

X_val = df_val.reset_index(drop=True)
X_val_dict = X_val.to_dict(orient='records')
X_val = dv.fit_transform(X_val_dict)

X_test = df_test.reset_index(drop=True)
X_test_dict = X_test.to_dict(orient='records')
X_test = dv.fit_transform(X_test_dict)

In [12]:
y_full_train = df_full_train['co2_emission_metrics_per_capita'].astype(int)
del df_full_train['co2_emission_metrics_per_capita']
dicts_full_train = df_full_train.to_dict(orient='records')

dv = DictVectorizer(sparse=False)
X_full_train = dv.fit_transform(dicts_full_train)

dicts_test = df_test.to_dict(orient='records')
X_test = dv.transform(dicts_test)

In [13]:
import xgboost as xgb
from xgboost import XGBClassifier
from xgboost.sklearn import XGBClassifier

In [14]:
dfulltrain = xgb.DMatrix(X_full_train, label=y_full_train,
                    feature_names=dv.get_feature_names())

dtest = xgb.DMatrix(X_test, feature_names=dv.get_feature_names())

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


##  Model Training

In [15]:

model = xgb.XGBRegressor(learning_rate = 0.5, n_estimators = 200, seed = 25)
 
model.fit(X_train, y_train)


[19:03:07] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


XGBRegressor(learning_rate=0.5, n_estimators=200, seed=25)

Save the `clf` model instance to BentoML local model store:

In [16]:
import bentoml

bentoml.sklearn.save_model("co2_emission_model", model)

Model(tag="co2_emission_model:e34hlcdamgvnuasc", path="/root/bentoml/models/co2_emission_model/e34hlcdamgvnuasc/")

Models saved can be accessed via `bentoml models` CLI command:

In [17]:
!bentoml models get co2_emission_model:latest

name: co2_emission_model                                                        
version: e34hlcdamgvnuasc                                                       
module: bentoml.sklearn                                                         
labels: {}                                                                      
options: {}                                                                     
metadata: {}                                                                    
context:                                                                        
  framework_name: sklearn                                                       
  framework_versions:                                                           
    scikit-learn: 1.0.2                                                         
  bentoml_version: 1.0.10                                                       
  python_version: 3.7.15                                                        
signatures:                 

In [18]:
!bentoml models list

 Tag                           Module           Size        Creation Time       
 co2_emission_model:e34hlcda…  bentoml.sklearn  108.26 KiB  2022-11-09 19:03:10 


To verify that the saved model can be loaded correctly:

In [19]:
loaded_model = bentoml.sklearn.load_model("co2_emission_model:latest")

loaded_model.predict(X_test)

[19:03:15] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


array([ 3.96991825e+00,  2.89582157e+00,  3.90133739e-01,  3.81041956e+00,
        7.35237718e-01,  1.01022310e+01,  3.85926306e-01,  5.76937103e+00,
        1.03421962e+00,  2.42962170e+00,  3.77210975e+00,  5.47820330e-02,
        2.36355114e+00,  2.96201897e+00,  4.91363585e-01,  3.15080762e-01,
        1.60844815e+00,  1.13750477e+01,  1.81739122e-01,  2.23340321e+00,
        2.95209646e-01,  1.23808831e-01,  2.43383265e+00,  6.60575724e+00,
        2.32693172e+00,  7.26609373e+00,  6.70532513e+00,  6.80411863e+00,
        1.93102336e+00,  1.34394312e+00,  8.80993271e+00, -2.00340080e+00,
        1.10505724e+01,  2.86899757e+00,  6.13670588e+00,  5.21996450e+00,
        9.46460056e+00,  2.20825043e+01,  2.13494635e+00,  4.05471039e+00,
        5.58597040e+00,  7.35179305e-01,  5.73042870e-01,  1.12821436e+01,
        6.76892185e+00,  1.33306766e+00,  9.70284939e+00,  1.76167507e+01,
        4.85361338e+00,  3.92721868e+00,  3.59889460e+00,  9.43729281e-03,
        7.82278967e+00,  

In [20]:
# Create a Runner instance:
model_runner = bentoml.sklearn.get("co2_emission_model:latest").to_runner()

# Runner#init_local initializes the model in current process, this is meant for development and testing only:
model_runner.init_local()

# This should yield the same result as the loaded model:
model_runner.predict.run(X_test)

[19:03:15] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


array([ 3.96991825e+00,  2.89582157e+00,  3.90133739e-01,  3.81041956e+00,
        7.35237718e-01,  1.01022310e+01,  3.85926306e-01,  5.76937103e+00,
        1.03421962e+00,  2.42962170e+00,  3.77210975e+00,  5.47820330e-02,
        2.36355114e+00,  2.96201897e+00,  4.91363585e-01,  3.15080762e-01,
        1.60844815e+00,  1.13750477e+01,  1.81739122e-01,  2.23340321e+00,
        2.95209646e-01,  1.23808831e-01,  2.43383265e+00,  6.60575724e+00,
        2.32693172e+00,  7.26609373e+00,  6.70532513e+00,  6.80411863e+00,
        1.93102336e+00,  1.34394312e+00,  8.80993271e+00, -2.00340080e+00,
        1.10505724e+01,  2.86899757e+00,  6.13670588e+00,  5.21996450e+00,
        9.46460056e+00,  2.20825043e+01,  2.13494635e+00,  4.05471039e+00,
        5.58597040e+00,  7.35179305e-01,  5.73042870e-01,  1.12821436e+01,
        6.76892185e+00,  1.33306766e+00,  9.70284939e+00,  1.76167507e+01,
        4.85361338e+00,  3.92721868e+00,  3.59889460e+00,  9.43729281e-03,
        7.82278967e+00,  

## Serving the model


In [21]:
%%writefile service.py
import numpy as np
import bentoml
from bentoml.io import NumpyNdarray

model_runner = bentoml.sklearn.get("co2_emission_model:latest").to_runner()

svc = bentoml.Service("co2_emission_classifier", runners=[model_runner])

@svc.api(input=NumpyNdarray(), output=NumpyNdarray())
def classify(input_series: np.ndarray) -> np.ndarray:
    return model_runner.predict.run(input_series)


Writing service.py


In [22]:
!bentoml serve service.py:svc --reload

2022-11-09T19:03:17+0000 [INFO] [cli] Prometheus metrics for HTTP BentoServer from "service.py:svc" can be accessed at http://localhost:3000/metrics.
2022-11-09T19:03:18+0000 [INFO] [cli] Starting development HTTP BentoServer from "service.py:svc" listening on http://0.0.0.0:3000 (Press CTRL+C to quit)
2022-11-09 19:03:20 circus[188] [INFO] Loading the plugin...
2022-11-09 19:03:20 circus[188] [INFO] Endpoint: 'tcp://127.0.0.1:51703'
2022-11-09 19:03:20 circus[188] [INFO] Pub/sub: 'tcp://127.0.0.1:39003'
2022-11-09T19:03:20+0000 [INFO] [observer] Watching directories: ['/content', '/root/bentoml/models']
[19:03:22] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


### Build Bento for deployment

In [23]:
%%writefile bentofile.yaml
service: "service.py:svc"
labels:
  owner: Andreeva_Arina
  project: Prediction of CO2 emission
include:
- "*.py"
python:
  packages:
    - scikit-learn
    - pandas

Writing bentofile.yaml


In [24]:
!pip install pydantic

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [25]:
from pydantic import BaseModel

In [26]:
# Create pydantic base class to create data schema for validation
class CreditApplication(BaseModel):
    Country : object
    Year : object
    GDP_Growth : float
    CO2_Emission_Metrics_per_capita : float
    Electricity_access : float
    Urban_population : float
    Population_growth : float
    Urban_population_growth_annual : float
    Live_stock_production : float
    Forest_area : float
    Renewable_energy_use : float
    Energy_use_kg_of_oil_per_capita : float
    Population_total : float

In [27]:
from bentoml.io import JSON

In [28]:
'''model_runner = bentoml.sklearn.get("co2_emission_model:latest").to_runner()

svc = bentoml.Service("co2_emission_classifier", runners=[model_runner])

@svc.api(input=JSON(pydantic_model=CreditApplication), output=JSON()) # decorate endpoint as in json format for input and output
def classify(credit_application):
    # transform pydantic class to dict to extract key-value pairs 
    application = credit_application.dict()
    # transform data from client using dictvectorizer
    vector = dv.transform(application)
    # make predictions using 'runner.predict.run(input)' instead of 'model.predict'
    prediction = model_runner.predict.run(vector) '''

'model_runner = bentoml.sklearn.get("co2_emission_model:latest").to_runner()\n\nsvc = bentoml.Service("co2_emission_classifier", runners=[model_runner])\n\n@svc.api(input=JSON(pydantic_model=CreditApplication), output=JSON()) # decorate endpoint as in json format for input and output\ndef classify(credit_application):\n    # transform pydantic class to dict to extract key-value pairs \n    application = credit_application.dict()\n    # transform data from client using dictvectorizer\n    vector = dv.transform(application)\n    # make predictions using \'runner.predict.run(input)\' instead of \'model.predict\'\n    prediction = model_runner.predict.run(vector) '

In [29]:
!bentoml build

Building BentoML service "co2_emission_classifier:hjdazltamgbh4asc" from build context "/content".
Packing model "co2_emission_model:e34hlcdamgvnuasc"
Locking PyPI package versions.
       ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.8/24.8 MB 46.5 MB/s eta 0:00:00
       ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.3/11.3 MB 82.9 MB/s eta 0:00:00
       ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 298.0/298.0 kB 21.6 MB/s eta 0:00:00
       ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 498.1/498.1 kB 31.5 MB/s eta 0:00:00
       ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.7/15.7 MB 43.5 MB/s eta 0:00:00
       ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 247.7/247.7 kB 19.8 MB/s eta 0:00:00
       ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.1/38.1 MB 25.8 MB/s eta 0:00:00

██████╗░███████╗███╗░░██╗████████╗░█████╗░███╗░░░███╗██╗░░░░░
██╔══██╗██╔════╝████╗░██║╚══██╔══╝██╔══██╗████╗░████║██║░░░░░
██████╦╝█████╗░░██╔██╗██║░░░██║░░░██║░░██║██╔████╔██║██║░░░░░
██╔══██╗██╔══╝░░██║╚████║░░░██║░░░██║░░██║██║╚██╔╝██║██║░░░░░
██

## Containerize and Deployment

In [30]:
!sudo apt install apt-transport-https ca-certificates curl software-properties-common -qq

ca-certificates is already the newest version (20211016~18.04.1).
curl is already the newest version (7.58.0-2ubuntu3.21).
software-properties-common is already the newest version (0.96.24.32.18).
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'sudo apt autoremove' to remove it.
The following NEW packages will be installed:
  apt-transport-https
0 upgraded, 1 newly installed, 0 to remove and 5 not upgraded.
Need to get 4,348 B of archives.
After this operation, 154 kB of additional disk space will be used.
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialog.pm line 76, <> line 1.)
debconf: falling back to frontend: Readline
debconf: unable to initialize frontend: Readline
debconf: (This frontend requires a controlling tty.)
debconf: falling back to frontend: Teletype
dpkg-preconfigure: unable t

In [31]:
!curl -fsSL https://download.docker.com/linux/ubuntu/gpg | sudo apt-key add -

OK


In [32]:
!sudo add-apt-repository "deb [arch=amd64] https://download.docker.com/linux/ubuntu bionic stable"

Hit:1 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:2 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:3 https://download.docker.com/linux/ubuntu bionic InRelease [64.4 kB]
Get:4 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Get:5 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Ign:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:8 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [83.3 kB]
Hit:9 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:10 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:11 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:12 https://download.docker.com/linux/ubuntu bionic/stable amd64 Packages [34.3 kB]
Hit:13 http://ppa.launchpad

In [ ]:
!sudo apt install docker-ce

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'sudo apt autoremove' to remove it.
The following additional packages will be installed:
  apparmor containerd.io dbus-user-session docker-ce-cli
  docker-ce-rootless-extras docker-scan-plugin iptables libip6tc0 libiptc0
  libnetfilter-conntrack3 libnfnetlink0 libxtables12 pigz
Suggested packages:
  apparmor-profiles-extra apparmor-utils aufs-tools cgroupfs-mount
  | cgroup-lite
Recommended packages:
  slirp4netns
The following NEW packages will be installed:
  apparmor containerd.io dbus-user-session docker-ce docker-ce-cli
  docker-ce-rootless-extras docker-scan-plugin iptables libip6tc0 libiptc0
  libnetfilter-conntrack3 libnfnetlink0 libxtables12 pigz
0 upgraded, 14 newly installed, 0 to remove and 8 not upgraded.
Need to get 103 MB of archives.
After this operation, 387 MB of addition

In [ ]:
!docker

In [ ]:
!bentoml containerize co2_emission_model:latest

In [ ]:
!docker run -p 3000:3000 co2_emission_model:3yult2takw6cwasc